In [6]:
import cirq
import random
import matplotlib.pyplot as plt
import numpy as np

In [23]:
def make_quantum_teleportation_circuit(gate):
    """Returns a circuit for quantum teleportation.

    This circuit 'teleports' a random qubit state prepared by
    the input gate from Alice to Bob.
    """
    circuit = cirq.Circuit()

    # Get the three qubits involved in the teleportation protocol.
    
    ##primero pertenece a Alice, el segundo también pertenece a Alice y es el que formaba parte del estado compartido, 
    ##y el tercero pertenecerá a Bob y forma parte del estado compartido
    
    alice = cirq.NamedQubit("Alice")
    msg = cirq.NamedQubit("Message")
    bob = cirq.NamedQubit("Bob")

    # The input gate prepares the message to send.
    circuit.append(gate(msg))

    # Create a Bell state shared between Alice and Bob.
    circuit.append([cirq.H(alice), cirq.CNOT(alice, bob)])

    # Bell measurement of the Message and Alice's entangled qubit.
    circuit.append([cirq.CNOT(msg, alice), cirq.H(msg), cirq.measure(msg, alice)])

    # Uses the two classical bits from the Bell measurement to recover the
    # original quantum message on Bob's entangled qubit.
    circuit.append([cirq.CNOT(alice, bob), cirq.CZ(msg, bob)])

    return circuit

In [24]:
"""Visualize the teleportation circuit."""
# Gate to put the message qubit in some state to send.
gate = cirq.X ** 0.25

# Create the teleportation circuit.
circuit = make_quantum_teleportation_circuit(gate)
print("Teleportation circuit:\n")
print(circuit)

Teleportation circuit:

Alice: ─────H────────@───X───────M───@───────
                     │   │       │   │
Bob: ────────────────X───┼───────┼───X───@───
                         │       │       │
Message: ───X^0.25───────@───H───M───────@───


In [44]:
"""Display the Bloch vector of the message qubit."""
message = cirq.Circuit(gate.on(cirq.NamedQubit("Message"))).final_state_vector()
message_bloch_vector = cirq.bloch_vector_from_state_vector(message, index=0)
print("Bloch vector of message qubit:")
print(np.round(message_bloch_vector, 3))

Bloch vector of message qubit:
[ 0.    -0.707  0.707]


In [50]:
"""Simulate the teleportation circuit and get the final state of Bob's qubit."""
# Get a simulator.
sim = cirq.Simulator()

# Simulate the teleportation circuit.
result = sim.simulate(circuit)

# Get the Bloch vector of Bob's qubit.
bobs_bloch_vector = cirq.bloch_vector_from_state_vector(result.final_state_vector, index=1)

print("Bloch vector of Bob's qubit:")
print(np.round(bobs_bloch_vector, 3))

message_bloch_vector2 = cirq.bloch_vector_from_state_vector(result.final_state_vector, index=0)
print("Bloch vector of Message's qubit despues de la teleportacion:")
print(np.round(message_bloch_vector2, 3))

##alice_bloch_vector2 = cirq.bloch_vector_from_state_vector(result.final_state_vector, index=2)
##print("Bloch vector of Alice's qubit:")
##print(np.round(alice_bloch_vector2, 3))

# Verify they are the same state!
np.testing.assert_allclose(bobs_bloch_vector, message_bloch_vector, atol=1e-7)


Bloch vector of Bob's qubit:
[ 0.    -0.707  0.707]
Bloch vector of Message's qubit:
[0. 0. 1.]


## Comentario:

Como podemos observar, los qubits de Bob despues de hacer la teleportacion cuantica, tiene el mismo estado que los qubits de Message antes de realizarlo. Por otra parte, tras realizar la teleportacion cuantica, Alice pierde su qubit ya que este qubit no puede estar en los dos sitios, ya que Bob tiene ahora el dicho qubit Y el qubit Message queda colapsado. La teleportacion cuantica, no se puede viajar mas a velocidad de luz.

Por otra, es importante mencionar que no se realiza una copia o clonacion del qubit original.